Preliminary work:
    - count 0s, 0.1s, and missing values

In [41]:
import synapseclient
import pandas as pd
import json

syn = synapseclient.Synapse()
syn.login(silent=True)

overall_scores = 'syn25905783'
overall_scores = syn.get(overall_scores)
overall_scores = pd.read_json(overall_scores.path, orient='records')


UPGRADE AVAILABLE

A more recent version of the Synapse Client (2.5.0) is available. Your version (2.4.0) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 2.5.0 release notes

https://python-docs.synapse.org/build/html/news.html



In [42]:
whatever = overall_scores[overall_scores['Logsdon'] <= 0.7165802586]
print(whatever['Logsdon'])

5158     0.616543
6299     0.378720
11050    0.539141
16850    0.716414
16851    0.716310
           ...   
28848    0.000000
28849    0.000000
28850    0.000000
28851    0.000000
28852    0.000000
Name: Logsdon, Length: 12006, dtype: float64
            Logsdon  GeneticsScore   OmicsScore  LiteratureScore
count  12006.000000   11215.000000  1929.000000      3309.000000
mean       0.254798       0.259970     0.046581         0.016226
std        0.179596       0.167199     0.145170         0.078864
min        0.000000       0.021895     0.000000         0.000000
25%        0.119926       0.134335     0.000000         0.000000
50%        0.224816       0.229439     0.000000         0.000000
75%        0.332893       0.329350     0.000000         0.000000
max        0.716414       0.716414     0.712887         0.640492


min and max of each field as column:

In [ ]:
# accumulate data in a list
# overall_scores.head()

In [48]:
for col in neo_matrix.keys():
    print(col)
    print(neo_matrix[col])

GeneticsScore
{'distribution': [8884, 5123, 3896, 3883, 3618, 1727, 472, 86, 13, 2], 'min': 0.022, 'max': 3.0, 'mean': 0.7512765364897127, 'bins': <zip object at 0x11bdc3540>, 'first_quartile': 0.2580353393, 'third_quartile': 1.17190550475}
OmicsScore
{'distribution': [8333, 2068, 1304, 1206, 1084, 887, 758, 553, 447, 392], 'min': 0.0, 'max': 2.0, 'mean': 0.5921848805057364, 'bins': <zip object at 0x11bdbb480>, 'first_quartile': 0.0, 'third_quartile': 1.1955014863}
LiteratureScore
{'distribution': [10067, 1579, 1231, 1228, 817, 808, 806, 806, 416, 389], 'min': 0.0, 'max': 2.0, 'mean': 0.4731542861502857, 'bins': <zip object at 0x11bd111c0>, 'first_quartile': 0.0, 'third_quartile': 0.879015384615385}


In [58]:
neo_matrix = {}
select_columns = overall_scores.columns[2:]
for col in select_columns:
    neo_matrix[col] = {}
    neo_matrix[col]["distribution"] = list(pd.cut(overall_scores[col], bins=10, precision=3).value_counts())
    neo_matrix[col]["min"] = round(overall_scores[col].min(), 3)
    neo_matrix[col]["max"] = round(overall_scores[col].max(), 3)
    neo_matrix[col]["mean"] = overall_scores[col].mean()
    discard, neo_matrix[col]["bins"] = list(pd.cut(overall_scores[col], bins=10, precision=3, retbins=True))
    neo_matrix[col]["bins"] = neo_matrix[col]["bins"].tolist()[1:]
    base = [0, neo_matrix[col]["bins"][:-1]]
    neo_matrix[col]["bins"] = zip(base, neo_matrix[col]["bins"])
    neo_matrix[col]["bins"] = list(zip(*neo_matrix[col]["bins"]))
    neo_matrix[col]["first_quartile"] = overall_scores[col].quantile(q=0.25, interpolation='midpoint')
    neo_matrix[col]["third_quartile"] = overall_scores[col].quantile(q=0.75, interpolation='midpoint')

In [61]:
print(neo_matrix)

{'Logsdon': {'distribution': [10961, 4424, 3915, 3383, 2638, 1722, 964, 611, 203, 32], 'min': 0.0, 'max': 5.978, 'mean': 1.368515886821121, 'bins': [(0, [0.59783573418, 1.19567146836, 1.79350720254, 2.39134293672, 2.9891786709000003, 3.58701440508, 4.18485013926, 4.78268587344, 5.38052160762]), (0.59783573418, 1.19567146836)], 'first_quartile': 0.2690120415, 'third_quartile': 2.2004251047}, 'GeneticsScore': {'distribution': [8884, 5123, 3896, 3883, 3618, 1727, 472, 86, 13, 2], 'min': 0.022, 'max': 3.0, 'mean': 0.7512765364897127, 'bins': [(0, [0.31970536697999996, 0.6175158817599999, 0.9153263965399999, 1.21313691132, 1.5109474261, 1.8087579408799999, 2.1065684556599997, 2.40437897044, 2.70218948522]), (0.31970536697999996, 0.6175158817599999)], 'first_quartile': 0.2580353393, 'third_quartile': 1.17190550475}, 'OmicsScore': {'distribution': [8333, 2068, 1304, 1206, 1084, 887, 758, 553, 447, 392], 'min': 0.0, 'max': 2.0, 'mean': 0.5921848805057364, 'bins': [(0, [0.2, 0.4, 0.600000000000

In [47]:
with open('../output/distribution_data_1.json', 'w') as outfile:
    json.dump(neo_matrix, outfile)